# Entrenamiento de Modelo Deep Learning con Nested Cross Validation

Este notebook implementa el entrenamiento de un modelo de Deep Learning para predicción de churn usando Nested Cross Validation y registra todo en MLflow.


In [21]:
# Importar librerías necesarias
import sys
from pathlib import Path

# Agregar el directorio raíz del proyecto al PATH
# El notebook está en notebooks/, necesitamos agregar el directorio padre (raíz del proyecto)
current_dir = Path.cwd()
if current_dir.name == 'notebooks':
    # Estamos en la carpeta notebooks, agregar el directorio padre
    project_root = current_dir.parent
else:
    # Estamos en la raíz, usar el directorio actual
    project_root = current_dir

# Agregar al PATH si no está ya
project_root_str = str(project_root)
if project_root_str not in sys.path:
    sys.path.insert(0, project_root_str)

print(f"✓ Directorio del proyecto agregado al PATH: {project_root}")

import pandas as pd
import numpy as np
import mlflow

# Ahora podemos importar desde src
from src.domain.services.preprocessing_service import PreprocessingService
from src.domain.models.deep_learning_model import DeepLearningModel
from src.application.use_cases.train_model_use_case import TrainModelUseCase
from src.infrastructure.mlflow.mlflow_tracking import MLflowTracking
from src.config.settings import settings

print("✓ Librerías importadas correctamente")
print(f"✓ MLflow Tracking URI: {settings.MLFLOW_TRACKING_URI}")
print(f"✓ MLflow Experiment: {settings.MLFLOW_EXPERIMENT_NAME}")


✓ Directorio del proyecto agregado al PATH: d:\GitHub\Nested_Learning_deeplearning
✓ Librerías importadas correctamente
✓ MLflow Tracking URI: http://localhost:5000
✓ MLflow Experiment: churn_prediction_nested_cv


## 1. Carga y Preparación de Datos


In [22]:
# Cargar dataset
df = pd.read_csv('../data/churn_data_clean.csv')

print(f"Dimensiones del dataset: {df.shape}")
print(f"\nPrimeras filas:")
df.head()


Dimensiones del dataset: (7042, 9)

Primeras filas:


,customerID,tenure,PhoneService,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,1,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,34,Yes,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,2,Yes,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,45,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,2,Yes,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [23]:
# Inicializar servicios
preprocessing_service = PreprocessingService()

# Preprocesar datos
X, y = preprocessing_service.preprocess_pipeline(df, fit=True)

print(f"Dimensiones de X: {X.shape}")
print(f"Dimensiones de y: {y.shape}")
print(f"\nDistribución de clases:")
unique, counts = np.unique(y, return_counts=True)
for u, c in zip(unique, counts):
    print(f"  Clase {u}: {c} ({c/len(y)*100:.2f}%)")


Dimensiones de X: (7042, 7)
Dimensiones de y: (7042,)

Distribución de clases:
  Clase 0: 5173 (73.46%)
  Clase 1: 1869 (26.54%)


## 2. Configuración de MLflow


In [24]:
# Configurar MLflow
settings.ensure_directories()

mlflow.set_tracking_uri(settings.MLFLOW_TRACKING_URI)
mlflow.set_experiment(settings.MLFLOW_EXPERIMENT_NAME)

mlflow_tracking = MLflowTracking()

print(f"Experimento configurado: {settings.MLFLOW_EXPERIMENT_NAME}")
print(f"Tracking URI: {mlflow.get_tracking_uri()}")

2026/01/02 02:25:10 INFO mlflow.tracking.fluent: Experiment with name 'churn_prediction_nested_cv' does not exist. Creating a new experiment.


Experimento configurado: churn_prediction_nested_cv
Tracking URI: http://localhost:5000


## 3. Definición de Hiperparámetros para Búsqueda


In [25]:
# Definir grilla de hiperparámetros para búsqueda
hyperparameter_grid = [
    {
        'hidden_layers': 2,
        'units_per_layer': [64, 32],
        'dropout_rate': 0.2,
        'learning_rate': 0.001,
        'batch_size': 32,
        'epochs': 50,
        'activation': 'relu',
        'optimizer': 'adam'
    },
    {
        'hidden_layers': 2,
        'units_per_layer': [128, 64],
        'dropout_rate': 0.3,
        'learning_rate': 0.001,
        'batch_size': 32,
        'epochs': 50,
        'activation': 'relu',
        'optimizer': 'adam'
    },
    {
        'hidden_layers': 3,
        'units_per_layer': [128, 64, 32],
        'dropout_rate': 0.3,
        'learning_rate': 0.0005,
        'batch_size': 64,
        'epochs': 50,
        'activation': 'relu',
        'optimizer': 'adam'
    },
    {
        'hidden_layers': 2,
        'units_per_layer': [256, 128],
        'dropout_rate': 0.4,
        'learning_rate': 0.0005,
        'batch_size': 64,
        'epochs': 50,
        'activation': 'relu',
        'optimizer': 'adam'
    }
]

print(f"Total de combinaciones de hiperparámetros: {len(hyperparameter_grid)}")
for i, hp in enumerate(hyperparameter_grid):
    print(f"\nConfiguración {i+1}:")
    for key, value in hp.items():
        print(f"  {key}: {value}")


Total de combinaciones de hiperparámetros: 4

Configuración 1:
  hidden_layers: 2
  units_per_layer: [64, 32]
  dropout_rate: 0.2
  learning_rate: 0.001
  batch_size: 32
  epochs: 50
  activation: relu
  optimizer: adam

Configuración 2:
  hidden_layers: 2
  units_per_layer: [128, 64]
  dropout_rate: 0.3
  learning_rate: 0.001
  batch_size: 32
  epochs: 50
  activation: relu
  optimizer: adam

Configuración 3:
  hidden_layers: 3
  units_per_layer: [128, 64, 32]
  dropout_rate: 0.3
  learning_rate: 0.0005
  batch_size: 64
  epochs: 50
  activation: relu
  optimizer: adam

Configuración 4:
  hidden_layers: 2
  units_per_layer: [256, 128]
  dropout_rate: 0.4
  learning_rate: 0.0005
  batch_size: 64
  epochs: 50
  activation: relu
  optimizer: adam


## 4. Entrenamiento con Nested Cross Validation

In [26]:
# Inicializar caso de uso
train_use_case = TrainModelUseCase(preprocessing_service)

# Configurar Nested CV
outer_k = 5  # Folds externos
inner_k = 3  # Folds internos para selección de hiperparámetros

print(f"Configuración de Nested Cross Validation:")
print(f"  - Folds externos (evaluación final): {outer_k}")
print(f"  - Folds internos (selección de hiperparámetros): {inner_k}")
print(f"\nIniciando entrenamiento...")


Configuración de Nested Cross Validation:
  - Folds externos (evaluación final): 5
  - Folds internos (selección de hiperparámetros): 3

Iniciando entrenamiento...


In [ ]:
import os
import warnings
import tensorflow as tf
import logging

# ------------------------------------------------------------------
# Configuración de warnings y logs
# ------------------------------------------------------------------

# TensorFlow: solo errores críticos
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"  # 0=ALL, 1=INFO, 2=WARNING, 3=ERROR

# Python warnings
warnings.filterwarnings(
    "ignore",
    category=DeprecationWarning
)

# Logging de TensorFlow
tf.get_logger().setLevel(logging.ERROR)
# Iniciar run principal en MLflow
with mlflow.start_run(run_name="nested_cv_training") as parent_run:
    # Registrar parámetros generales
    mlflow.log_params({
        "outer_cv_folds": outer_k,
        "inner_cv_folds": inner_k,
        "dataset_size": len(df),
        "features_count": X.shape[1],
        "hyperparameter_combinations": len(hyperparameter_grid)
    })
    
    # Ejecutar Nested Cross Validation
    results = train_use_case.nested_cross_validation(
        df=df,
        outer_k=outer_k,
        inner_k=inner_k,
        hyperparameter_grid=hyperparameter_grid,
        random_state=42
    )
    
    # Registrar métricas promedio
    mlflow.log_metrics(results['average_metrics'])
    
    # Registrar mejores hiperparámetros
    mlflow.log_params({
        f"best_{k}": v for k, v in results['best_hyperparameters'].items()
    })
    
    # Guardar y registrar el mejor modelo
    if results['best_model'] is not None:
        # Guardar modelo temporalmente
        model_path = "../models/best_churn_model"
        results['best_model'].save_model(model_path)
        
        # Registrar modelo en MLflow
        mlflow.keras.log_model(
            results['best_model'].model,
            artifact_path="model",
            registered_model_name=settings.MODEL_NAME
        )
        
        # Registrar en Model Registry con stage Production
        model_uri = f"runs:/{mlflow.active_run().info.run_id}/model"
        mlflow_tracking.register_model_version(
            model_uri=model_uri,
            registered_model_name=settings.MODEL_NAME,
            stage="Production"
        )
        
        print(f"\n✅ Modelo registrado en MLflow Model Registry:")
        print(f"   Nombre: {settings.MODEL_NAME}")
        print(f"   Stage: Production")
        print(f"   URI: {model_uri}")
    
    print("\n✅ Entrenamiento completado exitosamente")
    print(f"Resultados de la Nested CV:")
    print(f"  Métricas promedio: {results['average_metrics']}")
    print(f"  Mejores hiperparámetros: {results['best_hyperparameters']}")

Iniciando Nested Cross Validation: 5 folds externos, 3 folds internos

Fold Externo 1/5


## 5. Resultados del Entrenamiento


In [ ]:
# Mostrar resultados detallados
print("="*60)
print("RESUMEN DE RESULTADOS - NESTED CROSS VALIDATION")
print("="*60)

print("\n📊 MÉTRICAS PROMEDIO (en todos los folds externos):")
for metric, value in results['average_metrics'].items():
    print(f"   {metric}: {value:.4f}")

print("\n🔧 MEJORES HIPERPARÁMETROS:")
for key, value in results['best_hyperparameters'].items():
    print(f"   {key}: {value}")

print("\n📈 RESULTADOS POR FOLD EXTERNO:")
for fold_result in results['nested_cv_results']:
    print(f"\n   Fold {fold_result['fold']}:")
    for metric, value in fold_result['test_metrics'].items():
        print(f"      {metric}: {value:.4f}")


In [ ]:
# Visualizar resultados por fold
import matplotlib.pyplot as plt

fold_numbers = [r['fold'] for r in results['nested_cv_results']]
f1_scores = [r['test_metrics']['f1_score'] for r in results['nested_cv_results']]
accuracies = [r['test_metrics']['accuracy'] for r in results['nested_cv_results']]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(fold_numbers, f1_scores, marker='o', linewidth=2, markersize=8, label='F1-Score')
axes[0].axhline(y=results['average_metrics']['f1_score'], color='r', linestyle='--', label='Promedio')
axes[0].set_title('F1-Score por Fold Externo', fontweight='bold')
axes[0].set_xlabel('Fold')
axes[0].set_ylabel('F1-Score')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(fold_numbers, accuracies, marker='o', linewidth=2, markersize=8, label='Accuracy', color='green')
axes[1].axhline(y=results['average_metrics']['accuracy'], color='r', linestyle='--', label='Promedio')
axes[1].set_title('Accuracy por Fold Externo', fontweight='bold')
axes[1].set_xlabel('Fold')
axes[1].set_ylabel('Accuracy')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## 6. Guardar Preprocessing Service

Es importante guardar el preprocessing service para poder usarlo en producción.


In [ ]:
# Guardar preprocessing service usando pickle
import pickle
import joblib

preprocessing_path = "../models/preprocessing_service.pkl"
joblib.dump(preprocessing_service, preprocessing_path)

print(f"✅ Preprocessing service guardado en: {preprocessing_path}")

# También registrar como artefacto en MLflow
mlflow.log_artifact(preprocessing_path, "preprocessing")
print("✅ Preprocessing service registrado en MLflow")


## 7. Verificación del Modelo en MLflow

Verificar que el modelo se haya registrado correctamente en MLflow Model Registry.


In [ ]:
# Verificar modelo en MLflow
from mlflow.tracking import MlflowClient

client = MlflowClient(settings.MLFLOW_TRACKING_URI)

try:
    latest_version = client.get_latest_versions(settings.MODEL_NAME, stages=["Production"])[0]
    print(f"✅ Modelo encontrado en Model Registry:")
    print(f"   Nombre: {latest_version.name}")
    print(f"   Versión: {latest_version.version}")
    print(f"   Stage: {latest_version.current_stage}")
    print(f"   Run ID: {latest_version.run_id}")
    
    # Obtener información del run
    run = client.get_run(latest_version.run_id)
    print(f"\n📊 Métricas del modelo:")
    for key, value in run.data.metrics.items():
        print(f"   {key}: {value:.4f}")
        
except Exception as e:
    print(f"⚠️ Error al verificar modelo: {e}")


## 8. Conclusión

El modelo ha sido entrenado exitosamente usando Nested Cross Validation y registrado en MLflow. 

**Próximos pasos:**
1. El modelo está disponible en MLflow Model Registry
2. La API puede cargar el modelo desde MLflow
3. El frontend puede consumir la API para realizar predicciones

**Para usar el modelo:**
- Accede a MLflow UI: `http://localhost:5000`
- El modelo está registrado como: `churn_deep_learning_model`
- Stage: `Production`
